In [49]:
! pip install --upgrade google-cloud-speech google-cloud-texttospeech
! pip install sounddevice soundfile

Requirement already up-to-date: google-cloud-speech in /Users/adrianow/Documents/Programs/anaconda2/lib/python2.7/site-packages
    100% |████████████████████████████████| 51kB 191kB/s 
Requirement already up-to-date: google-api-core[grpc]<2.0.0dev,>=1.6.0 in /Users/adrianow/Documents/Programs/anaconda2/lib/python2.7/site-packages (from google-cloud-speech)
Requirement already up-to-date: six>=1.10.0 in /Users/adrianow/Documents/Programs/anaconda2/lib/python2.7/site-packages (from google-api-core[grpc]<2.0.0dev,>=1.6.0->google-cloud-speech)
Requirement already up-to-date: pytz in /Users/adrianow/Documents/Programs/anaconda2/lib/python2.7/site-packages (from google-api-core[grpc]<2.0.0dev,>=1.6.0->google-cloud-speech)
Requirement already up-to-date: googleapis-common-protos!=1.5.4,<2.0dev,>=1.5.3 in /Users/adrianow/Documents/Programs/anaconda2/lib/python2.7/site-packages (from google-api-core[grpc]<2.0.0dev,>=1.6.0->google-cloud-speech)
Requirement already up-to-date: futures>=3.2.0; py

In [102]:
import sounddevice as sd
import soundfile as sf
import numpy as np
import time

# define some parameters for recording
sr = 16000
sd.default.samplerate = sr
sd.default.channels = 1
sf.default_subtype('WAV')

def record(duration=3, sample_rate=sr, file_name='record.wav'):
    """
    Grava um áudio e salva em um arquivo
    """
    # grava o áudio salvando na variável data
    print("Gravando")
    data = sd.rec(duration * sample_rate, blocking=True)
#     print(sd.wait())
#     time.sleep(duration)
#     print(sd.wait())
    print("Pronto")
    
    # salva os dados
    sf.write(file_name, data, sample_rate)
    return file_name

def read(file_name):
    """ Read a file """
    return sf.read(file_name)

def play(data, sample_rate):
    sd.play(data, sample_rate, blocking=True)
    

fname = record()
data, fs = read(fname)
play(data, fs)

Gravando
Pronto


In [75]:
import io
import os

# pega o caminho até o arquivo que foi gravado
gae_key = 'gaekey.json'
curr_path = os.getcwd()
file_name = os.path.join(curr_path,gae_key)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = file_name

synth_file = 'synt_wave.wav'

In [90]:
# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types


def ouvir_o_comando():
    
    fname = record()
    
    # Instantiates a client
    client = speech.SpeechClient().from_service_account_json(
            'gaekey.json')

    # The name of the audio file to transcribe
    file_name = os.path.join(
        os.getcwd(),
        fname)

    # Loads the audio into memory
    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=sd.default.samplerate,
        language_code='pt-BR')

    # Detects speech in the audio file
    response = client.recognize(config, audio)

    for result in response.results:
        print(u'Transcript: {}'.format(result.alternatives[0].transcript))
        
    if response.results:
        return result.alternatives[0].transcript
    else:
        print(u'Sem retorno')
        return None
        
ouvir_o_comando()

Gravando
Pronto
Sem retorno


In [96]:
"""Synthesizes speech from the input string of text or ssml.

Note: ssml must be well-formed according to:
    https://www.w3.org/TR/speech-synthesis/
"""
from google.cloud import texttospeech

def tocar_texto(texto):
    print(u'Falando: {}'.format(texto))
    
    # Instantiates a client
    client = texttospeech.TextToSpeechClient()

    # Set the text input to be synthesized
    synthesis_input = texttospeech.types.SynthesisInput(text=texto)

    # Build the voice request, select the language code ("en-US") and the ssml
    # voice gender ("neutral")
    voice = texttospeech.types.VoiceSelectionParams(
        language_code='pt-BR',
        ssml_gender=texttospeech.enums.SsmlVoiceGender.MALE)

    # Select the type of audio file you want returned
    audio_config = texttospeech.types.AudioConfig(
        audio_encoding=texttospeech.enums.AudioEncoding.LINEAR16)

    # Perform the text-to-speech request on the text input with the selected
    # voice parameters and audio file type
    response = client.synthesize_speech(synthesis_input, voice, audio_config)

    # The response's audio_content is binary.
    with open(synth_file, 'wb') as out:
        # Write the response to the output file.
        out.write(response.audio_content)
        print('Audio content written to file {}'.format(synth_file))

    data, fs = read(synth_file)
    play(data,fs)
    
tocar_texto(texto = result.alternatives[0].transcript)

Falando: tratamento que uma coxinha já visse
Audio content written to file synt_wave.wav


In [140]:
def enum(**enums):
    return type('Enum', (), enums)

estados = enum(INICIO=0, BEM_VINDO=1, NAO_ENTENDI=2, TCHAU=3, PERGUNTA=4)

jarvis_texts = { 
    estados.INICIO: u'olá, posso saber o seu nome?',
    estados.BEM_VINDO: u'prazer em te conhecer, {aluno}. Como posso te ajudar?',
    estados.NAO_ENTENDI: u'ce tá ficando lôco!! Não faço isto. Me dê um outro comando!',
    estados.TCHAU: u'tchau {aluno}. Até a próxima',
    estados.PERGUNTA: u'Hahaha bom comando este seu {comando}. O que mais quer?'
}

transicoes = {
    estados.INICIO: estados.BEM_VINDO,
    estados.BEM_VINDO: estados.PERGUNTA,
    estados.PERGUNTA: estados.NAO_ENTENDI,
    estados.NAO_ENTENDI: estados.TCHAU
}

memoria = {
    
}

estado_atual = estados.INICIO
estado_antigo = None
insistencia = 0

while True:
    
    # busca o texto atual
    fala = jarvis_texts[estado_atual]
    
    # grava na memória se houver um detalhe necessário para o próximo estado
    comando_entrada = re.search(r'(\{.*\})', fala)
    if comando_entrada:
        key = comando_entrada.group()
        memoria[key] = memoria.get(key, comando)
        fala = fala.replace(key, memoria[key])
    
    # verifica se precisa falar o nome do Aluno
    tocar_texto(fala)
    
    # espere a resposta do Usuário
    comando = ouvir_o_comando()
    
    # verifica se a pessoa falou algo
    if not comando:
        estado_antigo = estado_atual
        estado_atual = estados.NAO_ENTENDI
        insistencia +=1 
    else:
        # verificar se pediu para sair ou não falou nada durante 3 segundos
        if comando=='sair' or insistencia==3:
            estado_antigo = estado_atual
            estado_atual = estados.TCHAU
        
        # verifica se o usuário pediu para sair ou se está falando algo
        if estado_atual==estados.TCHAU:
            key = '{aluno}'
            tocar_texto(jarvis_texts[estado_atual].replace(key, memoria[key]))
            break
        else:
            estado_antigo = estado_atual
            estado_atual = transicoes[estado_atual]
            insistencia = 0
    

Falando: olá, posso saber o seu nome?
Audio content written to file synt_wave.wav
Gravando
Pronto
Transcript: Adriano
Falando: prazer em te conhecer, Adriano. Como posso te ajudar?
Audio content written to file synt_wave.wav
Gravando
Pronto
Transcript: traz um café para mim
Falando: Hahaha bom comando este seu traz um café para mim. O que mais quer?
Audio content written to file synt_wave.wav
Gravando
Pronto
Transcript: sair
Falando: tchau Adriano. Até a próxima
Audio content written to file synt_wave.wav
